In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [4]:
# 재현성을 위한 시드 설정
np.random.seed(42)

def generate_raw_data(num_rows=1000):
    """
    지저분한(Dirty) 가상의 유저 행동 로그를 생성합니다.
    """
    data = []
    actions = ['click', 'view', 'purchase', 'add_to_cart', None] # None은 결측치 시뮬레이션
    
    start_time = datetime.now()
    
    for i in range(num_rows):
        user_id = random.randint(1000, 1100)
        action = random.choices(actions, weights=[40, 30, 10, 15, 5])[0] # 5% 확률로 결측치 발생
        
        # 가격 데이터 (이상치 포함: 음수 가격 혹은 터무니없이 비싼 가격)
        if action == 'purchase':
            price = round(random.uniform(10, 500), 2)
        else:
            price = 0
            
        # 1% 확률로 데이터 오염 (가격이 -100이거나 문자열이 섞임)
        if random.random() < 0.01:
            price = -100 
            
        timestamp = start_time - timedelta(minutes=random.randint(0, 10000))
        
        data.append([user_id, action, price, timestamp])
        
    df = pd.DataFrame(data, columns=['user_id', 'action', 'price', 'timestamp'])
    return df

# 데이터 생성
raw_df = generate_raw_data()
print(f"Original Data Size: {len(raw_df)}")
print(raw_df.head())

Original Data Size: 1000
   user_id       action  price                  timestamp
0     1059        click    0.0 2026-01-05 00:37:00.690902
1     1057        click    0.0 2026-01-02 04:40:00.690902
2     1000  add_to_cart    0.0 2026-01-03 21:27:00.690902
3     1021        click    0.0 2026-01-06 14:14:00.690902
4     1098         view    0.0 2026-01-08 14:24:00.690902


In [5]:
def preprocess_data(df):
    """
    AI 학습용 데이터로 만들기 위한 전처리(ETL의 Transform) 과정
    """
    df_clean = df.copy()
    
    # 1. 결측치(Null) 제거: 행동(action)이 없는 로그는 무의미하므로 삭제
    initial_count = len(df_clean)
    df_clean = df_clean.dropna(subset=['action'])
    print(f"Dropped {initial_count - len(df_clean)} rows with missing actions.")
    
    # 2. 이상치(Outlier) 처리: 가격이 0보다 작으면 데이터 오류로 간주하고 삭제
    # (실무에서는 평균값 대치 등을 쓰기도 하지만, 여기선 삭제로 진행)
    df_clean = df_clean[df_clean['price'] >= 0]
    
    # 3. 시간 순서 정렬 (시계열 데이터의 경우 중요)
    df_clean = df_clean.sort_values(by='timestamp').reset_index(drop=True)
    
    return df_clean

# 정제 실행
clean_df = preprocess_data(raw_df)
print(f"\nCleaned Data Size: {len(clean_df)}")
print(clean_df.describe()) # 통계 정보 확인

Dropped 46 rows with missing actions.

Cleaned Data Size: 942
           user_id       price                      timestamp
count   942.000000  942.000000                            942
mean   1049.686837   27.161677  2026-01-05 07:01:35.659054592
min    1000.000000    0.000000     2026-01-01 19:38:00.690902
25%    1025.000000    0.000000  2026-01-03 14:25:00.690902016
50%    1050.000000    0.000000  2026-01-05 09:25:00.690902016
75%    1076.000000    0.000000  2026-01-07 00:36:45.690902016
max    1100.000000  497.320000     2026-01-08 17:43:00.690902
std      29.518283   90.460026                            NaN


In [6]:
'''
과제 1. 비즈니스 로직(이상치 필터링) 추가하기
현재 코드는 음수 가격(price < 0)만 제거하고 있습니다. 하지만 현실에서는 실수로 입력된 너무 비싼 가격도 AI 모델의 학습을 방해(Gradient Explosion 등 유발)할 수 있습니다.

미션: preprocess_data 함수를 수정하여 가격(price)이 450을 초과하는 데이터도 '이상치'로 간주하여 삭제하세요.

목표: 단순한 오류(NULL) 처리뿐만 아니라, 도메인 지식에 기반한 데이터 클렌징을 경험합니다.
'''

business_df = clean_df[clean_df["price"] < 450]

business_df.head()
business_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 931 entries, 0 to 941
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_id    931 non-null    int64         
 1   action     931 non-null    object        
 2   price      931 non-null    float64       
 3   timestamp  931 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 36.4+ KB


In [7]:
'''
과제 2. AI를 위한 파생 변수(Feature) 생성하기
AI 모델은 2024-05-20 14:30:00 같은 타임스탬프 원본을 그대로 이해하지 못합니다. 보통 "오전/오후"나 "요일" 정보로 쪼개서 학습시킵니다.

미션: df_clean 데이터프레임에 is_weekend 라는 새로운 컬럼(Feature)을 만드세요.

timestamp가 토요일(5) 또는 일요일(6)이면 1, 평일이면 0을 넣습니다.

힌트: df['timestamp'].dt.weekday를 활용하세요.

목표: Data Engineering의 단계인 'Transform' 과정에서 AI 학습에 필요한 Feature를 미리 만들어주는 감각을 익힙니다.
'''


business_df['is_weekend'] = business_df['timestamp'].dt.weekday.map(lambda x: 1 if x >= 5 else 0)
business_df.head()

C:\Users\Sun\AppData\Local\Temp\ipykernel_1928\2690950561.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  business_df['is_weekend'] = business_df['timestamp'].dt.weekday.map(lambda x: 1 if x >= 5 else 0)


,user_id,action,price,timestamp,is_weekend
0,1079,view,0.00,2026-01-01 19:38:00.690902,0
1,1019,view,0.00,2026-01-01 19:39:00.690902,0
2,1075,click,0.00,2026-01-01 19:41:00.690902,0
3,1090,purchase,276.95,2026-01-01 20:15:00.690902,0
4,1020,view,0.00,2026-01-01 20:38:00.690902,0


In [ ]:
'''
과제 3. 데이터 정합성 검증 (Validation)
엔지니어링의 끝은 검증입니다. 위 1, 2번을 수행한 뒤 최종 데이터 상태를 확인하세요.

미션: 최종적으로 남은 데이터의 개수(len(df_clean))와, action 별 데이터 개수(value_counts())를 출력하세요.

질문: 과제 1번(450 초과 제거)을 적용하기 전과 후, 데이터 개수가 얼마나 줄어들었나요?
'''

print("지우기 전 개수:", len(clean_df))
print("지운 후 개수:", len(business_df))
print(business_df['action'].value_counts())

지우기 전 개수: 942
지운 후 개수: 931
action
click          399
view           282
add_to_cart    156
purchase        94
Name: count, dtype: int64
